In [1]:
!pip install --upgrade kiteconnect

  Running setup.py clean for kiteconnect
Failed to build kiteconnect
    Running setup.py install for kiteconnect: started
    Running setup.py install for kiteconnect: finished with status 'done'


  Command arguments: 'C:\Users\user\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-0gthmymx\\kiteconnect\\setup.py'"'"'; __file__='"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-0gthmymx\\kiteconnect\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\user\AppData\Local\Temp\pip-wheel-lpi7uw55'
  Command output: [use --verbose to show]
  DEPRECATION: kiteconnect was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. pip 21.0 will remove support for this functionality. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [4]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

In [ ]:
#requires automating for request token

In [5]:
API_KEY = "zh5pfekmc0zezour"
REQUEST_TOKEN = "lC1LkWdvEchxkuJt1O0KhAjs4VCG84WK"
API_SECRET = "rbbnk2qpp7be9i74sx27qb17ypv9qszh"

kite = KiteConnect(api_key=API_KEY)
# data = kite.generate_session(REQUEST_TOKEN, api_secret=API_SECRET)
# ACCESS_TOKEN = data["access_token"]
ACCESS_TOKEN = "vgfaU673sFRyvOsfGL0IpoUoWVhwSGKj"
kite.set_access_token(ACCESS_TOKEN)
print("Access Token is ",ACCESS_TOKEN)

Access Token is  vgfaU673sFRyvOsfGL0IpoUoWVhwSGKj


In [6]:
instr = kite.instruments(exchange="NSE")

In [9]:
for name in ['BHARTIARTL','BERGEPAINT','DRREDDY']:
    for inst in instr:
        if(name == inst['tradingsymbol']):
            print(inst)

OrderedDict([('instrument_token', 2714625), ('exchange_token', '10604'), ('tradingsymbol', 'BHARTIARTL'), ('name', 'BHARTI AIRTEL'), ('last_price', 0.0), ('expiry', ''), ('strike', 0.0), ('tick_size', 0.05), ('lot_size', 1), ('instrument_type', 'EQ'), ('segment', 'NSE'), ('exchange', 'NSE')])
OrderedDict([('instrument_token', 103425), ('exchange_token', '404'), ('tradingsymbol', 'BERGEPAINT'), ('name', 'BERGER PAINTS (I)'), ('last_price', 0.0), ('expiry', ''), ('strike', 0.0), ('tick_size', 0.05), ('lot_size', 1), ('instrument_type', 'EQ'), ('segment', 'NSE'), ('exchange', 'NSE')])
OrderedDict([('instrument_token', 225537), ('exchange_token', '881'), ('tradingsymbol', 'DRREDDY'), ('name', "DR. REDDY'S LABORATORIES"), ('last_price', 0.0), ('expiry', ''), ('strike', 0.0), ('tick_size', 0.05), ('lot_size', 1), ('instrument_type', 'EQ'), ('segment', 'NSE'), ('exchange', 'NSE')])


In [1]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import time
import pandas as pd

class CandleStick():
    def __init__(self):
        self.Open = 0
        self.High = 0
        self.Low = 0
        self.Close = 0
        
# Initialise
kws = KiteTicker("zh5pfekmc0zezour", "U8EEOgMXsDkOrhJF7uZdDpck4sIUQZuo")
kws._startedBefore = False

freq = 5

lastTime = float(int(int(time.strftime('%M', time.localtime(time.time()))) / freq) * freq)
#symbs = [738561]
#symbs = [404,10604,881]
symbs = []
names = ['BERGEPAINT','BHARTIARTL','DRREDDY']
API_KEY = "zh5pfekmc0zezour"
ACCESS_TOKEN = "vgfaU673sFRyvOsfGL0IpoUoWVhwSGKj"
kite = KiteConnect(api_key=API_KEY)
kite.set_access_token(ACCESS_TOKEN)
instr = kite.instruments(exchange="NSE")
for name in names:
    for inst in instr:
        if(name == inst['tradingsymbol']):
            symbs.append(inst['instrument_token'])
symbCand = []
symbDf = []
symbVol = [-1]*len(symbs)
for i in range(len(symbs)):
    symbCand.append(CandleStick())
    symbDf.append(
        pd.DataFrame(
            columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]))
    
newSymbVol = [0]*len(symbs)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    
    global lastTime, symbs, symbCand, symbDf, symbVol, names, newSymbVol
    
    myTicks = [None]*len(symbs)
    for i in range(len(symbs)):
        for tick in ticks:
            if(tick['instrument_token']==symbs[i]):
                myTicks[i] = tick

    tim = time.time()
    timFloat = float(time.strftime('%M', time.localtime(tim))) + (
        float(time.strftime('%S', time.localtime(tim))) / 60)
            
    if (timFloat < (lastTime + freq) and timFloat>= lastTime):
        
        for i in range(len(symbs)):
            
            currTick = myTicks[i]
            
            if(currTick != None):
                
                if(symbVol[i] == (-1)):
                    symbVol[i] = currTick['volume']
                    
                newSymbVol[i] = currTick['volume']
                
                ltp = currTick['last_price']
                symbCand[i].Close = ltp
                
                if (symbCand[i].Open == 0):
                    symbCand[i].Open = ltp
                    symbCand[i].High = ltp
                    symbCand[i].Low = ltp
                    
                if (ltp > symbCand[i].High):
                    symbCand[i].High = ltp
                    
                if (ltp < symbCand[i].Low):
                    symbCand[i].Low = ltp
    else:
        lastTime = (lastTime + freq)%60
            
        newSymbCand = []
        for i in range(len(symbs)):
            newSymbCand.append(CandleStick())
            symbDf[i] = symbDf[i].append(
                {
                    "Datetime":
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tim)),
                    "Open":
                    symbCand[i].Open,
                    "High":
                    symbCand[i].High,
                    "Low":
                    symbCand[i].Low,
                    "Close":
                    symbCand[i].Close,
                    "Volume":
                    newSymbVol[i] - symbVol[i]
                },
                ignore_index=True)
            print(names[i])
            print(symbDf[i].tail(1))
        symbCand = newSymbCand
        symbVol = newSymbVol[:]


def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    global symbs
    ws.subscribe(symbs)

    # Set RELIANCE to tick in `full` mode.
    ws.set_mode(ws.MODE_FULL, symbs)


def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()


# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

print("Started Simulation")

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

Started Simulation
BERGEPAINT
              Datetime    Open    High     Low  Close Volume
0  2020-11-12 12:45:00  648.65  648.65  647.85  648.0   1659
BHARTIARTL
              Datetime    Open   High    Low  Close Volume
0  2020-11-12 12:45:00  480.05  480.1  479.9  479.9  26600
DRREDDY
              Datetime    Open    High     Low    Close Volume
0  2020-11-12 12:45:00  4873.5  4874.0  4870.4  4871.55   3127
BERGEPAINT
              Datetime   Open    High    Low  Close Volume
1  2020-11-12 12:50:00  648.0  648.05  647.4  647.4   2938
BHARTIARTL
              Datetime   Open   High    Low   Close  Volume
1  2020-11-12 12:50:00  479.9  480.1  479.6  479.95  127062
DRREDDY
              Datetime     Open    High     Low    Close Volume
1  2020-11-12 12:50:00  4871.55  4884.0  4871.5  4878.05  18339
BERGEPAINT
              Datetime   Open    High     Low   Close Volume
2  2020-11-12 12:55:00  647.4  648.05  647.25  648.05   2576
BHARTIARTL
              Datetime    Open   High    Low 

Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Unhandled Error
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\defer.py", line 311, in addCallbacks
    self._runCallbacks()
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\base.py", line 447, in _continueFiring
    callable(*args, **kwargs)
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\base.py", line 706, in disconnectAll
    failure.Failure(main.CONNECTION_LOST))
--- <exception caught here> ---
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\python\log

In [1]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import time
import datetime
import pandas as pd

import os.path
from os import path

import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

tim = time.time()
timHrFloat = float(time.strftime('%H', time.localtime(tim))) + \
    (float(time.strftime('%M', time.localtime(tim))) / 60)

if(timHrFloat >= 15.27):
    print("Done for the Day")
    sys.exit()

names = ['BERGEPAINT', 'BHARTIARTL', 'DRREDDY']

API_KEY = "zh5pfekmc0zezour"

REQUEST_TOKEN = None
ACCESS_TOKEN = None

API_SECRET = "rbbnk2qpp7be9i74sx27qb17ypv9qszh"

def getRequestToken(API_KEY):
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    while(True):
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=1920x1080")

        driver = webdriver.Chrome(PATH, options=chrome_options)
        driver.get(
            "https://kite.trade/connect/login?api_key=" + API_KEY + "&v=3")

        time.sleep(1.5)

        formDiv = driver.find_element_by_class_name("login-form")
        loginBtn = driver.find_element_by_class_name(
            "actions").find_element_by_tag_name("button")
        loginInputs = formDiv.find_elements_by_tag_name("input")
        userIdInput = loginInputs[0]
        passwordInput = loginInputs[1]

        userIdInput.clear()
        passwordInput.clear()

        userIdInput.send_keys("GR6439")
        passwordInput.send_keys("Slackydoom123")
        loginBtn.click()

        time.sleep(2)

        pinInput = driver.find_element_by_tag_name("input")
        pinInput.clear()
        pinInput.send_keys("134679")
        driver.find_element_by_class_name(
            "actions").find_element_by_tag_name("button").click()

        time.sleep(4)

        currUrl = driver.current_url

        if "success" in currUrl:
            token = currUrl.split("request_token=")[1].split("&action=")[0]
            driver.quit()
            print("Access Token is ", token)
            return token
        else:
            driver.quit()
            print("Failure getting Access Token")
            time.sleep(8)


isInitPrintedWaiting = False

freq = 1  # in min

while True:

    tim = time.time()
    timHrFloat = float(time.strftime('%H', time.localtime(tim))) + \
        (float(time.strftime('%M', time.localtime(tim))) / 60)
    timFloat = float(time.strftime('%M', time.localtime(tim))) + \
        (float(time.strftime('%S', time.localtime(tim))) / 60)

    if((REQUEST_TOKEN == None) and timHrFloat >= 9.0):
        print("Fetching Tokens")
        # Replace with Selenium code to get Request Token
        REQUEST_TOKEN = getRequestToken(API_KEY)

        kite = KiteConnect(api_key=API_KEY)
        data = kite.generate_session(REQUEST_TOKEN, api_secret=API_SECRET)
        ACCESS_TOKEN = data["access_token"]
        kite.set_access_token(ACCESS_TOKEN)

    if((not (ACCESS_TOKEN == None)) and timHrFloat >= 9.25):
        if((int(timFloat*100) % (100 * freq)) < 10):
            break
        elif(not isInitPrintedWaiting):
            isInitPrintedWaiting = True
            print("Waiting to start")

# Initialise
kws = KiteTicker(API_KEY, ACCESS_TOKEN)
kws._startedBefore = False

lastTime = float(
    int(int(time.strftime('%M', time.localtime(time.time()))) / freq) * freq)

# symbs = [738561]
# symbs = [404,10604,881]
symbs = []

instr = kite.instruments(exchange="NSE")
for name in names:
    for inst in instr:
        if(name.replace("_", "&") == inst['tradingsymbol']):
            symbs.append(inst['instrument_token'])
symbDf = []
ticksDf = []
cells = [None]*len(symbs)

for i in range(len(symbs)):
    ticksDf.append(pd.DataFrame(
        columns=["timestamp", "ltp", "volume", "depth"]))
    csvName = ("KiteData/"+names[i]+".csv")

    if(path.exists(csvName)):
        symbDf.append(pd.read_csv(csvName))
    else:
        symbDf.append(
            pd.DataFrame(
                columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]))


def on_ticks(ws, ticks):
    # Callback to receive ticks.

    tim = time.time()
    
    timFloat = float(time.strftime('%M', time.localtime(tim))) + \
        (float(time.strftime('%S', time.localtime(tim))) / 60)
    timHrFloat = float(time.strftime('%H', time.localtime(tim))) + \
        (float(time.strftime('%M', time.localtime(tim))) / 60)

    global lastTime, symbs, symbDf, names, ticksDf, cells

    if(timHrFloat >= 15.27):
        print("Done for the Day")
        kws.close()

        for i in range(len(symbDf)):
            symbDf[i].to_csv(("KiteData/"+names[i]+".csv"), index=False)

    myTicks = [None]*len(symbs)
    for i in range(len(symbs)):
        for tick in ticks:
            if(tick['instrument_token'] == symbs[i]):
                myTicks[i] = tick

    if (timFloat < (lastTime + freq) and timFloat >= lastTime):

        for i in range(len(symbs)):

            currTick = myTicks[i]

            if(currTick != None):

                ltp = currTick['last_price']

                ticksDf[i] = ticksDf[i].append(
                    {"timestamp": currTick['timestamp'], "ltp": ltp, "volume": currTick['volume'], "depth": currTick['depth']}, ignore_index=True)

    else:

        currDt = datetime.datetime.now()
        currD = datetime.datetime(
            currDt.year, currDt.month, currDt.day, currDt.hour, currDt.minute)
        lastD = datetime.datetime(currDt.year, currDt.month, currDt.day,
                                  currDt.hour, currDt.minute) - datetime.timedelta(minutes=freq)

        lastTime = (lastTime + freq) % 60
        
        for i in range(len(symbs)):
            cells[i] = ticksDf[i][ticksDf[i]['timestamp'] >= lastD][ticksDf[i]
                                                                    ['timestamp'] < currD].sort_values(by=['timestamp'], ignore_index=True)

            symbDf[i] = symbDf[i].append(
                {
                    "Datetime":
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tim)),
                    "Open":
                    cells[i].ltp.head(1).iloc[0],
                    "High":
                    cells[i].ltp.max(),
                    "Low":
                    cells[i].ltp.min(),
                    "Close":
                    cells[i].ltp.tail(1).iloc[0],
                    "Volume":
                    cells[i].volume.tail(1).iloc[0] -
                    cells[i].volume.head(1).iloc[0]
                },
                ignore_index=True)

            print(names[i])
            
            print(symbDf[i].tail(1))

#             if(symbDf[i].shape[0] >= 4):
#                 # Take action instead
#                 print(symbDf[i].tail(1))
#             else:
#                 print("Still not of size greater than 3")


def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    global symbs
    ws.subscribe(symbs)

    # Set RELIANCE to tick in `full` mode.
    ws.set_mode(ws.MODE_FULL, symbs)


def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()


# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

print("Started Simulation")

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

Fetching Tokens
Access Token is  GxQsY6o5k3hngpVw53Z1C0vV2eM9y2IF
Waiting to start
Started Simulation


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


BERGEPAINT
              Datetime   Open   High    Low  Close Volume
0  2020-11-13 15:12:00  648.0  648.7  648.0  648.6   5159
BHARTIARTL
              Datetime    Open    High     Low   Close Volume
0  2020-11-13 15:12:00  474.25  474.85  474.25  474.85  91923
DRREDDY
              Datetime    Open    High      Low   Close Volume
0  2020-11-13 15:12:00  4845.2  4849.0  4844.15  4848.5   4654


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


BERGEPAINT
              Datetime   Open   High    Low  Close Volume
1  2020-11-13 15:13:00  648.7  649.2  648.6  649.2   2135
BHARTIARTL
              Datetime    Open    High     Low   Close Volume
1  2020-11-13 15:13:00  474.75  475.85  474.75  475.15  80073
DRREDDY
              Datetime     Open    High     Low    Close Volume
1  2020-11-13 15:13:00  4848.25  4855.0  4848.0  4854.95   6124


...

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


BERGEPAINT
              Datetime    Open    High    Low   Close Volume
3  2020-11-13 15:15:00  648.75  649.05  648.6  648.65   3605
BHARTIARTL
              Datetime   Open    High    Low  Close Volume
3  2020-11-13 15:15:00  474.9  475.45  474.8  475.2  37970
DRREDDY
              Datetime    Open    High     Low   Close Volume
3  2020-11-13 15:15:00  4857.2  4858.8  4854.7  4856.0   8225


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


BERGEPAINT
              Datetime   Open   High     Low   Close Volume
4  2020-11-13 15:16:00  648.7  649.0  648.65  648.85   2976
BHARTIARTL
              Datetime    Open    High    Low  Close Volume
4  2020-11-13 15:16:00  475.15  475.25  474.6  474.8  31388
DRREDDY
              Datetime    Open    High     Low   Close Volume
4  2020-11-13 15:16:00  4857.0  4858.0  4855.1  4858.0   4523
Done for the Day
BERGEPAINT
              Datetime   Open   High     Low   Close Volume
5  2020-11-13 15:17:00  648.9  649.3  648.85  649.25   5911
BHARTIARTL
              Datetime    Open    High     Low  Close Volume
5  2020-11-13 15:17:00  474.65  474.95  474.35  474.7  49841
DRREDDY
              Datetime    Open    High      Low   Close Volume
5  2020-11-13 15:17:00  4857.2  4862.7  4856.75  4860.0  13656


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
Connection closed: None - None


In [2]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import time
import datetime
import pandas as pd

import os.path
from os import path

import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

tim = time.time()
timHrFloat = float(time.strftime('%H', time.localtime(tim))) + \
    (float(time.strftime('%M', time.localtime(tim))) / 60)

if (timHrFloat >= 15.27):
    print("Done for the Day")
    sys.exit()

names = ['BERGEPAINT', 'BHARTIARTL', 'DRREDDY']

portfolioName = "InitPortfolio.txt"

if (path.exists(portfolioName)):
    portStr = ""
    with open(portfolioName, 'r') as f:
        for i in f.readlines():
            portStr += i  # string

    portfolio = eval(portStr)  # this is orignal dict with instace dict
else:
    portfolio = None

initLtps = {}


def initPortfolio(names, initLtps):
    portfolio = {
        "Initial Value": 0,
        "Current Value": 0,
        "Invested Value": 0,
        "Holdings": {},
        "Live Buy Orders": [],
        "Live Sell Orders": [],
        "Tickerwise Split": {},
        "Trades": {}
    }
    totalInitialValue = 0
    for name in names:
        portfolio["isInvested"][name] = False
        portfolio["Holdings"][name] = {"qty": 0, "ltp": None, "value": 0}
        portfolio["Trades"][name] = []
        curInitLtp = initLtps[name]
        if curInitLtp > 2000:
            curInitialValue = 1.1 * curInitLtp
        else:
            curInitialValue = 1.1 * 2000
        portfolio["Tickerwise Split"][name] = {
            "Initial Value": curInitialValue,
            "Current Value": curInitialValue
        }
        totalInitialValue += curInitialValue
    portfolio["Initial Value"] = totalInitialValue
    portfolio["Current Value"] = totalInitialValue
    return portfolio


feats = ['Close', 'Close1', 'Close2', 'Volume']

API_KEY = "zh5pfekmc0zezour"

REQUEST_TOKEN = None
ACCESS_TOKEN = None

API_SECRET = "rbbnk2qpp7be9i74sx27qb17ypv9qszh"


def getRequestToken(API_KEY):
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    while (True):
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=1920x1080")

        driver = webdriver.Chrome(PATH, options=chrome_options)
        driver.get("https://kite.trade/connect/login?api_key=" + API_KEY +
                   "&v=3")

        time.sleep(1.5)

        formDiv = driver.find_element_by_class_name("login-form")
        loginBtn = driver.find_element_by_class_name(
            "actions").find_element_by_tag_name("button")
        loginInputs = formDiv.find_elements_by_tag_name("input")
        userIdInput = loginInputs[0]
        passwordInput = loginInputs[1]

        userIdInput.clear()
        passwordInput.clear()

        userIdInput.send_keys("GR6439")
        passwordInput.send_keys("Slackydoom123")
        loginBtn.click()

        time.sleep(2)

        pinInput = driver.find_element_by_tag_name("input")
        pinInput.clear()
        pinInput.send_keys("134679")
        driver.find_element_by_class_name("actions").find_element_by_tag_name(
            "button").click()

        time.sleep(4)

        currUrl = driver.current_url

        if "success" in currUrl:
            token = currUrl.split("request_token=")[1].split("&action=")[0]
            driver.quit()
            print("Access Token is ", token)
            return token
        else:
            driver.quit()
            print("Failure getting Access Token")
            time.sleep(8)


def normalize(df0, tickerName):
    df_ticker = pd.DataFrame()
    df_ticker['Datetime'] = df0['Datetime']
    df_ticker['Ticker'] = tickerName
    df_ticker['Price'] = np.log(df0['Close'])
    df_ticker['Open'] = np.log(df0['Open']).diff()
    df_ticker['Open1'] = df_ticker['Open'].shift(1)
    df_ticker['Open2'] = df_ticker['Open'].shift(2)
    df_ticker['High'] = np.log(df0['High']).diff()
    df_ticker['High1'] = df_ticker['High'].shift(1)
    df_ticker['High2'] = df_ticker['High'].shift(2)
    df_ticker['Low'] = np.log(df0['Low']).diff()
    df_ticker['Low1'] = df_ticker['Low'].shift(1)
    df_ticker['Low2'] = df_ticker['Low'].shift(2)
    df_ticker['Close'] = np.log(df0['Close']).diff()
    df_ticker['Close1'] = df_ticker['Close'].shift(1)
    df_ticker['Close2'] = df_ticker['Close'].shift(2)
    df_ticker['Volume'] = np.array(df0['Volume']) / (pd.read_csv(
        (tickerName + ".csv"))['volume'].mean())
    df_ticker['Volume1'] = df_ticker['Volume'].shift(1)
    df_ticker['Volume2'] = df_ticker['Volume'].shift(2)
    df_ticker.drop(df_ticker.head(3).index, inplace=True)
    return df_ticker


isInitPrintedWaiting = False

while True:

    tim = time.time()
    timHrFloat = float(time.strftime('%H', time.localtime(tim))) + \
        (float(time.strftime('%M', time.localtime(tim))) / 60)
    timFloat = float(time.strftime('%M', time.localtime(tim))) + \
        (float(time.strftime('%S', time.localtime(tim))) / 60)

    if ((REQUEST_TOKEN == None) and timHrFloat >= 8.75):
        print("Fetching Tokens")
        # Replace with Selenium code to get Request Token
        REQUEST_TOKEN = getRequestToken(API_KEY)

        kite = KiteConnect(api_key=API_KEY)
        data = kite.generate_session(REQUEST_TOKEN, api_secret=API_SECRET)
        ACCESS_TOKEN = data["access_token"]
        kite.set_access_token(ACCESS_TOKEN)

        # Initialise
        kws = KiteTicker(API_KEY, ACCESS_TOKEN)
        kws._startedBefore = False

        symbs = []

        instr = kite.instruments(exchange="NSE")

        for name in names:
            for inst in instr:
                if (name.replace("_", "&") == inst['tradingsymbol']):
                    symbs.append(inst['instrument_token'])
        symbDf = []
        ticksDf = []
        cells = [None] * len(symbs)

        for i in range(len(symbs)):
            ticksDf.append(
                pd.DataFrame(columns=["timestamp", "ltp", "volume", "depth"]))
            csvName = ("KiteData/" + names[i] + ".csv")

            if (path.exists(csvName)):
                symbDf.append(pd.read_csv(csvName))
            else:
                symbDf.append(
                    pd.DataFrame(columns=[
                        "Datetime", "Open", "High", "Low", "Close", "Volume"
                    ]))

        weights = ''
        with open('Pen/164.txt', 'r') as f:
            for i in f.readlines():
                weights = i  # string

        weights = eval(weights)  # this is orignal dict with instace dict

        agent = getAgent(weights, feats)

    if ((not (ACCESS_TOKEN == None)) and timHrFloat >= 9.25):
        if ((int(timFloat * 100) % 500) < 10):
            break
        elif (not isInitPrintedWaiting):
            isInitPrintedWaiting = True
            print("Waiting for Take Off")

print("Initiating Take Off")

freq = 5

lastTime = float(
    int(int(time.strftime('%M', time.localtime(time.time()))) / freq) * freq)


def on_ticks(ws, ticks):
    # Callback to receive ticks.

    tim = time.time()
    timFloat = float(time.strftime('%M', time.localtime(tim))) + \
        (float(time.strftime('%S', time.localtime(tim))) / 60)
    timHrFloat = float(time.strftime('%H', time.localtime(tim))) + \
        (float(time.strftime('%M', time.localtime(tim))) / 60)

    global lastTime, symbs, symbDf, names, ticksDf, cells, portfolio

    if (timHrFloat >= 15.27):
        print("Done for the Day")
        with open(portfolioName, 'w+') as f:
            f.write(str(portfolio))
        for i in range(len(symbDf)):
            symbDf[i].to_csv(("KiteData/" + names[i] + ".csv"), index=False)
        kws.close()

    myTicks = [None] * len(symbs)
    for i in range(len(symbs)):
        for tick in ticks:
            if (tick['instrument_token'] == symbs[i]):
                myTicks[i] = tick

    if (timFloat < (lastTime + freq) and timFloat >= lastTime):

        for i in range(len(symbs)):

            currTick = myTicks[i]

            if (currTick != None):

                if (portfolio == None):
                    if (len(initLtps) < len(symbs)):
                        initLtps[names[i]] = currTick['last_price']
                    else:
                        portfolio = initPortfolio(names, initLtps)

                ticksDf[i] = ticksDf[i].append(
                    {
                        "timestamp": currTick['timestamp'],
                        "ltp": currTick['last_price'],
                        "volume": currTick['volume'],
                        "depth": currTick['depth']
                    },
                    ignore_index=True)
    else:
        currDt = datetime.datetime.now()
        currD = datetime.datetime(currDt.year, currDt.month, currDt.day,
                                  currDt.hour, currDt.minute)
        lastD = datetime.datetime(
            currDt.year, currDt.month, currDt.day, currDt.hour,
            currDt.minute) - datetime.timedelta(minutes=freq)
        lastTime = (lastTime + freq) % 60

        newBuyOrders = []
        newSellOrders = []

        for i in range(len(symbs)):

            cells[i] = ticksDf[i][ticksDf[i]['timestamp'] >= lastD][
                ticksDf[i]['timestamp'] < currD].sort_values(by=['timestamp'],
                                                             ignore_index=True)

            if (cells[i].shape[0] == 0):
                print("No ticks received for ", names[i], " at ",
                      currDt.minute, ".")
            else:
                symbDf[i] = symbDf[i].append(
                    {
                        "Datetime":
                        time.strftime('%Y-%m-%d %H:%M:%S',
                                      time.localtime(tim)),
                        "Open":
                        cells[i].ltp.head(1).iloc[0],
                        "High":
                        cells[i].ltp.max(),
                        "Low":
                        cells[i].ltp.min(),
                        "Close":
                        cells[i].ltp.tail(1).iloc[0],
                        "Volume":
                        cells[i].volume.tail(1).iloc[0] -
                        cells[i].volume.head(1).iloc[0]
                    },
                    ignore_index=True)

                if (not portfolio == None):
                    for buyOrder in portfolio["Live Buy Orders"]:
                        if (buyOrder["ticker"] == names[i]):
                            for j in range(cells[i].shape[0]):
                                nu = cells[i].iloc[j]
                                bids = nu["depth"]["buy"]
                                for bid in bids:
                                    if (buyOrder["qty"] <= bid["quantity"]
                                            and portfolio["Holdings"][
                                                names[i]]["qty"] == 0):
                                        if (portfolio['Current Value'] >=
                                            (buyOrder["qty"] * bid["price"])):
                                            print("Bought ", buyOrder["qty"],
                                                  " of ", names[i], " at ",
                                                  nu["timestamp"], " for ",
                                                  bid["price"])
                                            portfolio["Trades"][
                                                names[i]].append({
                                                    "Type":
                                                    "Buy",
                                                    "Time":
                                                    nu["timestamp"],
                                                    "Quantity":
                                                    buyOrder["qty"],
                                                    "Price":
                                                    bid["price"]
                                                })
                                            portfolio['Current Value'] -= (
                                                buyOrder["qty"] * bid["price"])
                                            portfolio['Tickerwise Split'][
                                                names[i]]["Current Value"] -= (
                                                    buyOrder["qty"] *
                                                    bid["price"])
                                            portfolio["Holdings"][names[i]] = {
                                                "qty":
                                                buyOrder["qty"],
                                                "ltp":
                                                cells[i].tail(1).ltp.iloc[0],
                                                "value":
                                                (buyOrder["qty"] *
                                                 cells[i].tail(1).ltp.iloc[0])
                                            }
                                        else:
                                            print(
                                                "Unable to buy ",
                                                buyOrder["qty"], " of ",
                                                names[i], " at ",
                                                nu["timestamp"], " for ",
                                                bid["price"],
                                                " due to insufficient funds.")
                                if (not portfolio["Holdings"][names[i]]["qty"]
                                        == 0):
                                    break
                    for sellOrder in portfolio["Live Sell Orders"]:
                        if (sellOrder["ticker"] == names[i]):
                            for j in range(cells[i].shape[0]):
                                nu = cells[i].iloc[j]
                                bids = nu["depth"]["sell"]
                                for bid in bids:
                                    if (sellOrder["qty"] <= bid["quantity"]
                                            and (not portfolio["Holdings"][
                                                names[i]]["qty"] == 0)):
                                        print("Sold ", sellOrder["qty"],
                                              " of ", names[i], " at ",
                                              nu["timestamp"], " for ",
                                              bid["price"])
                                        portfolio["Trades"][names[i]].append({
                                            "Type":
                                            "Sell",
                                            "Time":
                                            nu["timestamp"],
                                            "Quantity":
                                            sellOrder["qty"],
                                            "Price":
                                            bid["price"]
                                        })
                                        portfolio['Current Value'] += (
                                            sellOrder["qty"] * bid["price"])
                                        portfolio['Tickerwise Split'][
                                            names[i]]["Current Value"] += (
                                                sellOrder["qty"] *
                                                bid["price"])
                                        portfolio["Holdings"][names[i]] = {
                                            "qty": 0,
                                            "ltp":
                                            cells[i].tail(1).ltp.iloc[0],
                                            "value": 0
                                        }
                                if (portfolio["Holdings"][names[i]]["qty"] == 0
                                    ):
                                    break

                if (symbDf[i].shape[0] >= 4):
                    # Take action instead
                    state = normalize(symbDf[i], names[i])[feats].values[-1]
                    signal = agent.act(state)
                    if (signal == 0):
                        if (portfolio["Holdings"][names[i]]["qty"] == 0):
                            print(
                                "Buy Signal for ", names[i], " at ",
                                time.strftime('%Y-%m-%d %H:%M:%S',
                                              time.localtime(tim)))
                            curBuyQty = int(
                                portfolio["Tickerwise Split"][
                                    names[i]]["Current Value"] /
                                (1.1 * cells[i].tail(1).ltp.iloc[0]))
                            if curBuyQty > 0:
                                newBuyOrders.append({
                                    "ticker": names[i],
                                    "qty": curBuyQty
                                })
                    elif (signal == 1):
                        if (portfolio["Holdings"][names[i]]["qty"] > 0):
                            print(
                                "Sell Signal for ", names[i], " at ",
                                time.strftime('%Y-%m-%d %H:%M:%S',
                                              time.localtime(tim)))
                            newSellOrders.append({
                                "ticker":
                                names[i],
                                "qty":
                                portfolio["Holdings"][names[i]]["qty"]
                            })
                else:
                    print("Still not of size greater than 3")

        portfolio["Live Buy Orders"] = newBuyOrders
        portfolio["Live Sell Orders"] = newSellOrders
        curInvestedValue = 0
        for holding in portfolio["Holdings"].values():
            curInvestedValue += holding["value"]
        portfolio["Invested Value"] = curInvestedValue


def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    global symbs
    ws.subscribe(symbs)

    # Set RELIANCE to tick in `full` mode.
    ws.set_mode(ws.MODE_FULL, symbs)


def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()


# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

print("Maara has Taken Off")

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

Fetching Tokens
Access Token is  wPzAAmiS4Gv4aEksPlFsDc3vb2ubvxMx
Waiting for Take Off
Initiating Take Off
Maara has Taken Off
BERGEPAINT
Still not of size greater than 3
BHARTIARTL
Still not of size greater than 3
DRREDDY
Still not of size greater than 3


Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Unhandled Error
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\defer.py", line 311, in addCallbacks
    self._runCallbacks()
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\base.py", line 447, in _continueFiring
    callable(*args, **kwargs)
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\internet\base.py", line 706, in disconnectAll
    failure.Failure(main.CONNECTION_LOST))
--- <exception caught here> ---
  File "C:\Users\user\Anaconda3\lib\site-packages\twisted\python\log

In [7]:
isBought = False
for i in range(cells[0].shape[0]):
    nu = cells[0].iloc[i]
    bids = nu["depth"]["buy"]
    for bid in bids:
        if (qty >= bid["quantity"] and not isBought):
            print("Bought at ", nu["timestamp"], " for ", bid["price"])
            isBought = True
    if(isBought):
        break

timestamp                                  2020-11-17 14:20:00
ltp                                                      644.4
volume                                                  569959
depth        {'buy': [{'quantity': 45, 'price': 644.2, 'ord...
Name: 0, dtype: object


In [19]:
tria = ""
tria += "DI"
tria

'DI'

In [36]:
isSold = False

def sell(nu, qty):
    global isSold
    bids = nu["depth"]["sell"]
    for bid in bids:
        if (qty >= bid["quantity"] and not isSold):
            print("Sold at ", nu["timestamp"], " for ", bid["price"])
            isSold = True

cells[0].apply(sell, axis=1, qty=250)

print("Done")

Sold at  2020-11-17 11:05:01  for  644.4
Done


In [16]:
drio = []
drio.append(2)
drio

[2]

In [3]:
portfolio = {
    "Initial Value": 0,
    "Current Value": 0,
    "Holdings": [{
        "ticker": "DRREDDY",
        "qty": 1
    }],
    "Live Buy Orders": [{
        "ticker": "RELIANCE",
        "qty": 3
    }],
    "Live Sell Orders": [{
        "ticker": "DRREDDY",
        "qty": 1
    }],
    "Tickerwise Split": {
        "RELIANCE": {
            "Initial Value": 0,
            "Current Value": 0
        },
        "DRREDDY": {
            "Initial Value": 0,
            "Current Value": 0
        }
    }
}


def initPortfolio(names, initLtps):
    portfolio = {
        "Initial Value": 0,
        "Current Value": 0,
        "Holdings": [],
        "Live Buy Orders": [],
        "Live Sell Orders": [],
        "Tickerwise Split": {}
    }
    totalInitialValue = 0
    for name in names:
        curInitLtp = initLtps[name]
        if curInitLtp > 2000:
            curInitialValue = 1.1 * curInitLtp
        else:
            curInitialValue = 1.1 * 2000
        portfolio["Tickerwise Split"][name] = {
            "Initial Value": curInitialValue,
            "Current Value": curInitialValue
        }
        totalInitialValue += curInitialValue
    portfolio["Initial Value"] = totalInitialValue
    portfolio["Current Value"] = totalInitialValue
    return portfolio

len(initPortfolio(names, {'BERGEPAINT': 600, 'BHARTIARTL': 400, 'DRREDDY': 3000}))

6

In [1]:
def getAgent(weights, feats):
    tickers = [
        'OBEROIRLTY', 'ONGC', 'OIL', 'OFSS', 'ORIENTCEM', 'PCJEWELLER',
        'ORIENTELEC', 'PIIND', 'PNBHOUSING', 'PERSISTENT', 'PNCINFRA',
        'PFIZER', 'PETRONET', 'PTC', 'PHILIPCARB', 'PVR', 'PHOENIXLTD', 'PFC',
        'PAGEIND', 'PIDILITIND', 'PARAGMILK', 'POWERGRID', 'PEL', 'PRAJIND',
        'PRESTIGE', 'POLYCAB', 'PRSMJOHNSN', 'PGHH', 'QUESS', 'PGHL', 'PNB',
        'RBLBANK', 'RECLTD', 'RADICO', 'RITES', 'RVNL', 'RAIN', 'RAJESHEXPO',
        'RALLIS', 'RCF', 'SHK', 'RAYMOND', 'SBILIFE', 'REDINGTON', 'SJVN',
        'RELAXO', 'SKFINDIA', 'RELCAPITAL', 'RELIANCE', 'SRF', 'SADBHAV',
        'RELINFRA', 'RPOWER', 'SANOFI', 'SIS', 'REPCOHOME', 'SHRIRAMCIT',
        'SRTRANSFIN', 'SHILPAMED', 'SOBHA', 'SIEMENS', 'SHOPERSTOP',
        'SPICEJET', 'SBIN', 'SHREECEM', 'STARCEMENT', 'SONATSOFTW', 'RENUKA',
        'SAIL', 'SWSOLAR', 'SUNPHARMA', 'SPARC', 'SOUTHBANK', 'STRTECH',
        'SUNTV', 'STAR', 'SUNDARMFIN', 'SUDARSCHEM', 'SUNDRMFAST', 'SUPRAJIT',
        'SUPREMEIND', 'SUNTECK', 'SYMPHONY', 'SUZLON', 'SYNGENE', 'TCIEXP',
        'TTKPRESTIG', 'TVTODAY', 'TV18BRDCST', 'TCS', 'TVSMOTOR', 'TATACONSUM',
        'TAKE', 'TATAELXSI', 'TATAINVEST', 'TATASTLBSL', 'TATAMOTORS',
        'TATASTEEL', 'TATAPOWER', 'TATAMTRDVR', 'TECHM', 'TEAMLEASE', 'NIACL',
        'RAMCOCEM', 'THYROCARE', 'TIMETECHNO', 'THERMAX', 'TIMKEN',
        'TORNTPHARM', 'TITAN', 'TORNTPOWER', 'TRIDENT', 'TRENT', 'UCOBANK',
        'TIINDIA', 'UFLEX', 'UPL', 'UJJIVAN', 'ULTRACEMCO', 'UNIONBANK',
        'VAIBHAVGBL', 'MCDOWELL-N', 'UBL', 'VGUARD', 'VAKRANGEE', 'VMART',
        'VTL', 'VIPIND', 'VBL', 'VARROC', 'VRLLOG', 'VEDL', 'VSTIND', 'IDEA',
        'VENKEYS', 'WABAG', 'VOLTAS', 'VINATIORGA', 'WHIRLPOOL', 'ZEEL',
        'ZENSARTECH', 'WIPRO', 'WELCORP', 'ZYDUSWELL', 'WOCKPHARMA',
        'WELSPUNIND', 'ECLERX', 'WESTLIFE', '3MINDIA', 'AIAENG', 'APLAPOLLO',
        'AUBANK', 'AARTIIND', 'AAVAS', 'ADANIGREEN', 'ABBOTINDIA',
        'ADANIPORTS', 'ADANITRANS', 'ADANIGAS', 'ABCAPITAL', 'ADANIPOWER',
        'ADVENZYMES', 'AJANTPHARM', 'ABFRL', 'AEGISCHEM', 'ALKEM', 'AFFLE',
        'APLLTD', 'ALLCARGO', 'AMBER', 'APOLLOHOSP', 'AMARAJABAT', 'AMBUJACEM',
        'APOLLOTYRE', 'ASHOKLEY', 'ARVINDFASN', 'ASHOKA', 'ASIANPAINT',
        'ASTRAL', 'ASTERDM', 'AUROPHARMA', 'ASTRAZEN', 'ATUL', 'AVANTIFEED',
        'BASF', 'DMART', 'BEML', 'AXISBANK', 'BSE', 'BALKRISIND', 'BAJAJ-AUTO',
        'BALMLAWRIE', 'BAJAJCON', 'BALRAMCHIN', 'BAJAJELEC', 'BAJFINANCE',
        'BANDHANBNK', 'BAJAJFINSV', 'BANKBARODA', 'BAJAJHLDNG', 'BANKINDIA',
        'MAHABANK', 'BERGEPAINT', 'BHARTIARTL', 'BATAINDIA', 'BDL', 'INFRATEL',
        'BAYERCROP', 'BEL', 'BIOCON', 'BIRLACORPN', 'BHARATFORG', 'BLUEDART',
        'BLUESTARCO', 'BHEL', 'BSOFT', 'BBTC', 'BPCL', 'BLISSGVS', 'BOMDYEING',
        'BRIGADE', 'CCL', 'BOSCHLTD', 'BRITANNIA', 'CESC', 'CARERATING',
        'CANFINHOME', 'CRISIL', 'CANBK', 'CAPLIPOINT', 'CADILAHC',
        'CASTROLIND', 'CEATLTD', 'CARBORUNIV', 'CENTRALBK', 'CDSL',
        'CENTURYPLY', 'CHALET', 'CHAMBLFERT', 'CHENNPETRO', 'COALINDIA',
        'CHOLAHLDNG', 'COCHINSHIP', 'CHOLAFIN', 'COLPAL', 'CIPLA', 'DCBBANK',
        'CONCOR', 'DBCORP', 'CUB', 'COROMANDEL', 'DCMSHRIRAM', 'DLF',
        'CREDITACC', 'CROMPTON', 'DABUR', 'DEEPAKNTR', 'CUMMINSIND',
        'DALBHARAT', 'DELTACORP', 'CYIENT', 'DBL', 'DHFL', 'DISHTV',
        'DIVISLAB', 'DCAL', 'DIXON', 'EIDPARRY', 'LALPATHLAB', 'EIHOTEL',
        'DRREDDY', 'EMAMILTD', 'EDELWEISS', 'ENDURANCE', 'ENGINERSIN',
        'EICHERMOT', 'EQUITAS', 'ERIS', 'FEDERALBNK', 'ESCORTS', 'FINEORG',
        'FINCABLES', 'EXIDEIND', 'FINPIPE', 'FCONSUMER', 'FDC', 'FLFL', 'FSL',
        'FRETAIL', 'FORTIS', 'GAIL', 'GEPIL', 'GMRINFRA', 'GARFIBRES',
        'GALAXYSURF', 'GHCL', 'GICRE', 'GLAXO', 'GILLETTE', 'GLENMARK',
        'GODREJCP', 'GODFRYPHLP', 'GODREJAGRO', 'GODREJIND', 'GRANULES',
        'GODREJPROP', 'GESHIP', 'GRAPHITE', 'GRASIM', 'GREAVESCOT', 'GMDCLTD',
        'GUJALKALI', 'GNFC', 'GSPL', 'GUJGASLTD', 'GPPL', 'GSFC', 'HEG',
        'HDFCLIFE', 'HCLTECH', 'HFCL', 'HEIDELBERG', 'HDFCAMC', 'HAVELLS',
        'HDFCBANK', 'HEROMOTOCO', 'HEXAWARE', 'HSCL', 'HIMATSEIDE', 'HINDALCO',
        'HAL', 'HINDCOPPER', 'HINDPETRO', 'HINDUNILVR', 'HINDZINC', 'HONAUT',
        'HDFC', 'HUDCO', 'ICICIGI', 'ICICIBANK', 'ICICIPRULI', 'ISEC', 'IDBI',
        'IFCI', 'IDFCFIRSTB', 'IIFL', 'IDFC', 'IRB', 'IFBIND', 'IRCON',
        'ITDCEM', 'ITC', 'INDIACEM', 'ITI', 'IBULHSGFIN', 'IBREALEST',
        'IBVENTURES', 'INDIANB', 'INDHOTEL', 'INDIAMART', 'IEX', 'IOC', 'IGL',
        'IOB', 'INDUSINDBK', 'NAUKRI', 'INFIBEAM', 'INFY', 'INOXLEISUR',
        'INDIGO', 'IPCALAB', 'JBCHEPHARM', 'JKCEMENT', 'JKLAKSHMI', 'JAGRAN',
        'JKPAPER', 'JAICORPLTD', 'JKTYRE', 'JISLJALEQS', 'JMFINANCIL',
        'J_KBANK', 'JSWENERGY', 'JAMNAAUTO', 'JSWSTEEL', 'JINDALSAW',
        'JUSTDIAL', 'JSLHISAR', 'JUBLFOOD', 'JSL', 'JYOTHYLAB', 'JUBILANT',
        'JINDALSTEL', 'KRBL', 'KANSAINER', 'KAJARIACER', 'KEI', 'KTKBANK',
        'KNRCON', 'KALPATPOWR', 'KEC', 'KARURVYSYA', 'KPITTECH', 'KSCL',
        'KOTAKBANK', 'KOLTEPATIL', 'L_TFH', 'LTTS', 'LICHSGFIN', 'LAKSHVILAS',
        'LTI', 'LT', 'LAURUSLABS', 'LEMONTREE', 'MAHSCOOTER', 'LUXIND',
        'LINDEINDIA', 'MAHSEAMLES', 'LUPIN', 'MMTC', 'M_MFIN', 'MAHINDCIE',
        'MARICO', 'MOIL', 'M_M', 'MHRIL', 'MARUTI', 'MRF', 'MFSL', 'MAHLOG',
        'MINDTREE', 'METROPOLIS', 'MAGMA', 'MANAPPURAM', 'MGL', 'MINDAIND',
        'MINDACORP', 'MRPL', 'MOTILALOFS', 'MIDHANI', 'MUTHOOTFIN', 'MPHASIS',
        'NATCOPHARM', 'NCC', 'MOTHERSUMI', 'MCX', 'NBCC', 'NESCO', 'NHPC',
        'NLCINDIA', 'NMDC', 'NTPC', 'NH', 'NATIONALUM', 'NFL', 'NBVENTURES',
        'NAVINFLUOR', 'NILKAMAL', 'NESTLEIND', 'NETWORK18', 'NAM-INDIA'
    ]

    Ntest = 1600
    train_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for ticker in tickers:
        df_ticker = pd.DataFrame()
        tickerFileName = ticker + ".csv"
        df0 = pd.read_csv(tickerFileName).iloc[750:-1].rename(columns={
            "open": "Open", "high": "High", "close": "Close", "low": "Low", "volume": "Volume"})
        df_ticker['Datetime'] = df0['Datetime']
        df_ticker['Ticker'] = ticker
        df_ticker['Price'] = np.log(df0['Close'])
        df_ticker['Open'] = np.log(df0['Open']).diff()
        df_ticker['Open1'] = df_ticker['Open'].shift(1)
        df_ticker['Open2'] = df_ticker['Open'].shift(2)
        df_ticker['High'] = np.log(df0['High']).diff()
        df_ticker['High1'] = df_ticker['High'].shift(1)
        df_ticker['High2'] = df_ticker['High'].shift(2)
        df_ticker['Low'] = np.log(df0['Low']).diff()
        df_ticker['Low1'] = df_ticker['Low'].shift(1)
        df_ticker['Low2'] = df_ticker['Low'].shift(2)
        df_ticker['Close'] = np.log(df0['Close']).diff()
        df_ticker['Close1'] = df_ticker['Close'].shift(1)
        df_ticker['Close2'] = df_ticker['Close'].shift(2)
        df_ticker['Volume'] = np.array(df0['Volume'])/(df0['Volume'].mean())
        df_ticker['Volume1'] = df_ticker['Volume'].shift(1)
        df_ticker['Volume2'] = df_ticker['Volume'].shift(2)
        df_ticker.drop(df_ticker.head(3).index, inplace=True)
        train_data = train_data.append(df_ticker.iloc[:-Ntest], ignore_index=True)
        test_data = test_data.append(df_ticker.iloc[-Ntest:], ignore_index=True)
    
    class Env:
        def __init__(self, df):
            self.df = df.reset_index()
            self.n = len(self.df)
            self.current_idx = 0
            self.action_space = [0, 1, 2]  # BUY, SELL, HOLD
            self.invested = 0

            self.states = self.df[feats].values
            self.rewards = self.df['Close'].values

        def reset(self):
            self.current_idx = 0
            self.invested = 0
            return self.states[self.current_idx]

        def step(self, action):
            # need to return (next_state, reward, done)

            self.current_idx += 1
            if self.current_idx >= self.n:
                raise Exception("Episode already done")

            if action == 0 and self.invested==0:  # BUY
                self.invested = 1
            elif action == 1 and self.invested==1:  # SELL
                self.invested = 0

            # compute reward

            if self.invested:
                reward = self.rewards[self.current_idx]
            else:
                reward = 0

            # state transition
            next_state = self.states[self.current_idx]

            done = (self.current_idx == self.n - 1)
            return next_state, reward, done
    
    class StateMapper:
        def __init__(self, env, n_bins=20, n_samples=10000):
            # first, collect sample states from the environment
            states = []
            done = False
            s = env.reset()
            self.D = len(s)  # number of elements we need to bin
            states.append(s)
            for _ in range(n_samples):
                a = np.random.choice(env.action_space)
                s2, _, done = env.step(a)
                states.append(s2)
                if done:
                    s = env.reset()
                    states.append(s)

            # convert to numpy array for easy indexing
            states = np.array(states)

            # create the bins for each dimension
            self.bins = []
            for d in range(self.D):
                column = np.sort(states[:, d])

                # find the boundaries for each bin
                current_bin = []
                for k in range(n_bins):
                    boundary = column[int(n_samples / n_bins * (k + 0.5))]
                    current_bin.append(boundary)

                self.bins.append(current_bin)

        def transform(self, state):
            x = np.zeros(self.D)
            for d in range(self.D):
                x[d] = int(np.digitize(state[d], self.bins[d]))
            return tuple(x)

        def all_possible_states(self):
            list_of_bins = []
            for d in range(self.D):
                list_of_bins.append(list(range(len(self.bins[d]) + 1)))
            # print(list_of_bins)
            return itertools.product(*list_of_bins)

    class Agent:
        def __init__(self, action_size, state_mapper, weights):
            self.action_size = action_size
            self.gamma = 0.9  # discount rate
            self.epsilon = 0
            self.learning_rate = 1e-1
            self.state_mapper = state_mapper

            # initialize Q-table randomly
            self.Q = weights

        def act(self, state):
            if np.random.rand() <= self.epsilon:
                return np.random.choice(self.action_size)

            s = self.state_mapper.transform(state)
            act_values = [self.Q[(s, a)] for a in range(self.action_size)]
            return np.argmax(act_values)  # returns action

        def train(self, state, action, reward, next_state, done):
            s = self.state_mapper.transform(state)
            s2 = self.state_mapper.transform(next_state)

            if done:
                target = reward
            else:
                act_values = [self.Q[(s2, a)] for a in range(self.action_size)]
                target = reward + self.gamma * np.amax(act_values)

            # Run one training step
            self.Q[(s, action)] += self.learning_rate * \
                (target - self.Q[(s, action)])
            
    train_env = Env(train_data)

    action_size = len(train_env.action_space)
    state_mapper = StateMapper(train_env)
    return Agent(action_size, state_mapper, weights)

In [4]:
from kiteconnect import KiteConnect

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [38]:
def getRequestToken():
    PATH = "C:\Program Files (x86)\chromedriver.exe"
    while(True):
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=1920x1080")
        
        driver = webdriver.Chrome(PATH, options=chrome_options)
        driver.get("https://kite.trade/connect/login?api_key=zh5pfekmc0zezour&v=3")

        time.sleep(1)

        formDiv = driver.find_element_by_class_name("login-form")
        loginBtn = driver.find_element_by_class_name("actions").find_element_by_tag_name("button")
        loginInputs = formDiv.find_elements_by_tag_name("input")
        userIdInput = loginInputs[0]
        passwordInput = loginInputs[1]

        userIdInput.clear()
        passwordInput.clear()

        userIdInput.send_keys("GR6439")
        passwordInput.send_keys("Slackydoom123")
        loginBtn.click()

        time.sleep(2)

        pinInput = driver.find_element_by_tag_name("input")
        pinInput.clear()
        pinInput.send_keys("134679")
        driver.find_element_by_class_name("actions").find_element_by_tag_name("button").click()

        time.sleep(4)

        currUrl = driver.current_url

        if "success" in currUrl:
            token = currUrl.split("request_token=")[1].split("&action=")[0]
            driver.quit()
            return token
        else:
            driver.quit()
            time.sleep(5)

In [39]:
getRequestToken()

'av5MO4v2JtjH1EFP0d9JqoFqYPRjMn0L'

In [33]:
driver.current_url.split("request_token=")[1].split("&action=")[0]

'i5nwIhPgAi6jeBafVFTLFbKoSLTYDJBL'

In [16]:
pd.DataFrame(
    columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]).shape[0]

0

In [5]:
myTick[0]['volume']

15701622

In [7]:
myTick[0]['last_price']

2012.15

In [8]:
import time

In [13]:
tim = time.time()
float(time.strftime('%M', time.localtime(tim))) +(float(time.strftime('%S', time.localtime(tim)))/60)

17.366666666666667

In [20]:
class CandleStick():
    def __init__(self):
        self.Open = None
        self.High = None
        self.Low = None

In [15]:
import pandas as pd

In [19]:
pd.DataFrame(columns=["Open","High","Low","Close","Volume"])

,Open,High,Low,Close


In [ ]:
lastTime = float(int(int(time.strftime('%M', time.localtime(tim)))/5)*5)
symbs = [738561]
symbCand = []
symbDf = []
symbVol = []
for i in range(len(symbs)):
    symbCand.append(CandleStick())
    symbDf.appenda(
        pd.DataFrame(
            columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]))

tim = time.time()
timFloat = float(time.strftime('%M', time.localtime(tim))) + (
    float(time.strftime('%S', time.localtime(tim))) / 60)
if (len(symbVol) == 0):
    for i in range(len(symbs)):
        symbVol.append(ticks[i]['volume'])
if (timFloat < (lastTime + 5)):
    for i in range(len(symbs)):
        ltp = ticks[i]['last_price']
        if (symbCand[i].Open == None):
            symbCand[i].Open = ltp
            symbCand[i].High = ltp
            symbCand[i].Low = ltp
        if (ltp > symbCand[i].High):
            symbCand[i].High = ltp
        if (ltp < symbCand[i].Low):
            symbCand[i].Low = ltp
else:
    if (not lastTime == 55):
        lastTime += 5
    else:
        lastTime = 0
    newSymbVol = []
    newSymbCand = []
    for i in range(len(symbs)):
        newSymbVol.append(ticks[i]['volume'])
        newSymbCand.append(CandleStick())
        symbDf[i].append(
            {
                "Datetime": time.strftime('%Y-%m-%d %H:%M:%S',
                                          time.localtime(tim)),
                "Open": symbCand[i].Open,
                "High": symbCand[i].High,
                "Low": symbCand[i].Low,
                "Close": ticks[i]['last_price'],
                "Volume": newSymbVol[i] - symbVol[i]
            },
            ignore_index=True)
        print(i,symbDf[i])
    symbCand = newSymbCand
    symbVol = newSymbVol

In [27]:
float(int(int(time.strftime('%M', time.localtime(tim)))/5)*5)

15.0

In [25]:
int(3.8)

3

In [28]:
3.0+2

5.0

In [30]:
55==55.0

True